<a href="https://colab.research.google.com/github/asetya/BigData/blob/master/ImageClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

praktikum mesin learning untuk image recognition 

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import mahotas
import cv2
import os
import h5py


jika ada pesan error ngga ditemukan mahotas jalankan kode instalasi mahotas di bahan ini (note - buang tanda #) 


```
!pip install mahotas
```



In [ ]:
#!pip install mahotas

     |████████████████████████████████| 5.7MB 2.2MB/s 


Konek ke googl drive kita


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Copy Dataset dari lokal ke server google colab

In [ ]:
!cp /content/drive/My\ Drive/BIMBINGAN/Naskah_Thesis/MuhammadNafiuddin/imageclassification/dataset1.zip /content/

**unzip dataset di server google colab**

In [ ]:
!unzip /content/dataset1.zip

Archive:  /content/dataset1.zip
  inflating: dataset2/desktop.ini    
   creating: dataset2/test/
  inflating: dataset2/test/1.jpg     
  inflating: dataset2/test/2.jpg     
  inflating: dataset2/test/3.jpg     
  inflating: dataset2/test/76.jpg    
  inflating: dataset2/test/77.jpg    
  inflating: dataset2/test/78.jpg    
  inflating: dataset2/test/79.jpg    
  inflating: dataset2/test/80.jpg    
  inflating: dataset2/test/desktop.ini  
   creating: dataset2/train/
   creating: dataset2/train/bluebell/
  inflating: dataset2/train/bluebell/1.jpg  
  inflating: dataset2/train/bluebell/10.jpg  
  inflating: dataset2/train/bluebell/11.jpg  
  inflating: dataset2/train/bluebell/12.jpg  
  inflating: dataset2/train/bluebell/13.jpg  
  inflating: dataset2/train/bluebell/14.jpg  
  inflating: dataset2/train/bluebell/15.jpg  
  inflating: dataset2/train/bluebell/16.jpg  
  inflating: dataset2/train/bluebell/17.jpg  
  inflating: dataset2/train/bluebell/18.jpg  
  inflating: dataset2/train/blu

In [ ]:
images_per_class = 80
fixed_size       = tuple((500, 500))
train_path       = "dataset2/train"
h5_data          = 'output/data.h5'
h5_labels        = 'output/labels.h5'
bins             = 8


feture descriptor 

In [ ]:
def fd_hu_moments(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    feature = cv2.HuMoments(cv2.moments(image)).flatten()
    return feature

panggil feature detector hu moment 

In [ ]:
#!mkdir output

**fungsi untuk mencari haralick feature**

In [ ]:
def fd_haralick(image):
    # convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # compute the haralick texture feature vector
    haralick = mahotas.features.haralick(gray).mean(axis=0)
    # return the result
    return haralick


**Fungsi untuk mencari histofram**

In [ ]:
def fd_histogram(image, mask=None):
    # convert the image to HSV color-space
    image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    # compute the color histogram
    hist  = cv2.calcHist([image], [0, 1, 2], None, [bins, bins, bins], [0, 256, 0, 256, 0, 256])
    # normalize the histogram
    cv2.normalize(hist, hist)
    # return the histogram
    return hist.flatten()

**MNgambil label training **
Melakukan Training 


In [ ]:
train_labels = os.listdir(train_path)

In [ ]:
print(train_labels)
# sort the training labels
train_labels.sort()
print(train_labels)

# empty lists to hold feature vectors and labels
global_features = []
labels          = []

['coltsfoot', 'desktop.ini', 'bluebell', 'buttercup']
['bluebell', 'buttercup', 'coltsfoot', 'desktop.ini']


In [ ]:
for training_name in train_labels:
  dir = os.path.join(train_path, training_name)
  if os.path.isdir(dir):
    print(dir)
    print(" adalah folder")
    current_label = training_name
    for x in range(1,images_per_class+1):
          # get the image file name
          file = dir + "/" + str(x) + ".jpg"

          # read the image and resize it to a fixed-size
          image = cv2.imread(file)
          image = cv2.resize(image, fixed_size)

          ####################################
          # Global Feature extraction
          ####################################
          fv_hu_moments = fd_hu_moments(image)
          fv_haralick   = fd_haralick(image)
          fv_histogram  = fd_histogram(image)

          ###################################
          # Concatenate global features
          ###################################
          global_feature = np.hstack([fv_histogram, fv_haralick, fv_hu_moments])

          # update the list of labels and feature vectors
          labels.append(current_label)
          global_features.append(global_feature)
   
    print("[STATUS] processed folder: {}".format(current_label))
 
# get the overall feature vector size
print("[STATUS] feature vector size {}".format(np.array(global_features).shape))

# get the overall training label size
print("[STATUS] training Labels {}".format(np.array(labels).shape))

# encode the target labels
targetNames = np.unique(labels)
le          = LabelEncoder()
target      = le.fit_transform(labels)
print("[STATUS] training labels encoded...")

# scale features in the range (0-1)
scaler            = MinMaxScaler(feature_range=(0, 1))
rescaled_features = scaler.fit_transform(global_features)
print("[STATUS] feature vector normalized...")

print("[STATUS] target labels: {}".format(target))
print("[STATUS] target labels shape: {}".format(target.shape))

# save the feature vector using HDF5
h5f_data = h5py.File(h5_data, 'w')
h5f_data.create_dataset('dataset_1', data=np.array(rescaled_features))

h5f_label = h5py.File(h5_labels, 'w')
h5f_label.create_dataset('dataset_1', data=np.array(target))

h5f_data.close()
h5f_label.close()

print("[STATUS] end of training..")    


  



dataset2/train/bluebell
 adalah folder
[STATUS] processed folder: bluebell
dataset2/train/buttercup
 adalah folder
[STATUS] processed folder: buttercup
dataset2/train/coltsfoot
 adalah folder
[STATUS] processed folder: coltsfoot
[STATUS] feature vector size (480, 532)
[STATUS] training Labels (480,)
[STATUS] training labels encoded...
[STATUS] feature vector normalized...
[STATUS] target labels: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 

**Loop Training untuk seluruh sub folder **


In [ ]:

import glob
import warnings
from matplotlib import pyplot
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.externals import joblib
warnings.filterwarnings('ignore')

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)
